# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms, positions)
pystruct.make_supercell([2, 2, 2])
lattice   = load_lattice(pystruct)
positions = load_positions(pystruct)
atoms     = fill(Si, length(positions));

Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_x+lda_c_pw, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
                               ElementPsp(Si, psp="hgh/lda/si-q4")
                               ElementPsp(Si, psp="hgh/lda/si-q4")
                               ElementPsp(Si, psp="hgh/lda/si-q4")
                               ElementPsp(Si, psp="hgh/lda/si-q4")
                               ElementPsp(Si, psp="hgh/lda/si-q4")
                               E

Find the ground state using direct minimisation and newton
(always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis; tol=1e-3);
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres.ψ, scfres.occupation)
scfres_newton = newton(basis, ψ; tol=1e-12)

Iter     Function value   Gradient norm 
     0     1.126946e+02     1.355756e+00
 * time: 0.5855238437652588
     1     1.002602e+01     9.554448e-01
 * time: 2.1121950149536133
     2    -1.222838e+01     9.511734e-01
 * time: 2.2183940410614014
     3    -3.450598e+01     6.868746e-01
 * time: 2.3704938888549805
     4    -4.797763e+01     5.038862e-01
 * time: 2.5492749214172363
     5    -5.712718e+01     2.043575e-01
 * time: 2.681128978729248
     6    -5.993966e+01     1.222688e-01
 * time: 2.780697822570801
     7    -6.097354e+01     4.531792e-02
 * time: 2.8960318565368652
     8    -6.137262e+01     6.018563e-02
 * time: 2.9925408363342285
     9    -6.165684e+01     4.036611e-02
 * time: 3.0910048484802246
    10    -6.187130e+01     2.577009e-02
 * time: 3.196712017059326
    11    -6.202819e+01     1.814572e-02
 * time: 3.3022890090942383
    12    -6.209134e+01     1.411881e-02
 * time: 3.3986120223999023
    13    -6.215552e+01     1.326732e-02
 * time: 3.4945089817047

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1139), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =  1139), [0.0, 0.14054984958423578, 0.5621993983369431, 1.264948646258122, 2.2487975933477724, 3.513746239605895, 3.513746239605895, 2.2487975933477724, 1.264948646258122, 0.5621993983369431  …  1.1243987966738864, 2.014547844040713, 3.185796590576011, 4.638145036279781, 4.12279558780425, 2.7641470418233043, 1.6865981950108295, 0.8901490473668268, 0.37479959889129544, 0.14054984958423578]), DFTK.RealSpaceMultiplication{Float64,

In [5]:
scfres_newton.energies

Energy breakdown (in Ha):
    Kinetic             25.7697876
    AtomicLocal         -18.8616201
    AtomicNonlocal      14.8540134
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8508714 
    Xc                  -19.3506430

    total               -62.277715861368